In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

model = tf.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    return normalized_result

directory_path = 'D:/fashion_canadian_images'

# Get list of filenames in the directory
filenames = []

for file in os.listdir(directory_path):
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        filenames.append(os.path.join(directory_path, file))

feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file, model))

pickle.dump(feature_list, open('embeddings.pkl', 'wb'))
pickle.dump(filenames, open('filenames.pkl', 'wb'))

100%|███████████████████████████████████████████████████████████████████████████████████| 2686/2686 [05:42<00:00,  7.85it/s]


In [2]:
!pip install tqdm

     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00


In [39]:
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2

feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

img = image.load_img('samples/.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

distances,indices = neighbors.kneighbors([normalized_result])

print(indices)

for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)


[[1741 2680 2606 2681 2682 2672]]


In [9]:
!pip install opencv-python


   ---------------------------------------- 38.1/38.1 MB 22.6 MB/s eta 0:00:00


In [46]:
img = image.load_img('samples/tshirt5.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

neighbors = NearestNeighbors(n_neighbors=10,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

distances,indices = neighbors.kneighbors([normalized_result])

print(indices)

for file in indices[0][1:10]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)


[[1766 2678 2681 2684 1741 2680   95   41   98 2676]]


In [36]:
import pandas as pd
import os
import shutil

# Path to the Excel file containing image names
excel_file_path = 'image_names.xlsx'

# Source folder containing the images
source_folder = 'D:/fashion_canadian_images/'

# Specify the range of rows you want to extract
start_row = 1770
end_row = 6077

# Read the Excel file
data = pd.read_excel(excel_file_path)

# Extract the specified rows
selected_rows = data.iloc[start_row-1:end_row]

# Create a destination folder
destination_folder = 'D:/temp_fashion/'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move the selected image files to the destination folder
for index, row in selected_rows.iterrows():
    image_name = row['ImageNameColumn']  # Replace 'ImageNameColumn' with the actual column name in your Excel file
    
    source_file_path = os.path.join(source_folder, image_name)
    destination_file_path = os.path.join(destination_folder, image_name)
    
    if os.path.exists(source_file_path):
        shutil.move(source_file_path, destination_file_path)
        print(f"Moved {image_name} to {destination_folder}")
    else:
        print(f"File {image_name} not found in {source_folder}")

print("Files moved successfully!")

XLRDError: Excel xlsx file; not supported

In [37]:
import openpyxl
import os
import shutil

# Path to the Excel file containing image names
excel_file_path = 'image_names.xlsx'

# Source folder containing the images
source_folder = 'D:/fashion_canadian_images/'

# Specify the range of rows you want to extract
start_row = 1770
end_row = 6077

# Load the Excel file
workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

# Create a destination folder
destination_folder = 'D:/temp_fashion/'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move the selected image files to the destination folder
for row_number in range(start_row, end_row + 1):
    image_name = sheet.cell(row=row_number, column=1).value  # Assuming image names are in the first column
    
    source_file_path = os.path.join(source_folder, image_name)
    destination_file_path = os.path.join(destination_folder, image_name)
    
    if os.path.exists(source_file_path):
        shutil.move(source_file_path, destination_file_path)
        print(f"Moved {image_name} to {destination_folder}")
    else:
        print(f"File {image_name} not found in {source_folder}")

print("Files moved successfully!")

Moved 20231130235218_01e94d24_row1771.jpg to D:/temp_fashion/
Moved 20231130235218_6cad836b_row1772.jpg to D:/temp_fashion/
Moved 20231130235219_74f2e12c_row1773.jpg to D:/temp_fashion/
Moved 20231130235220_c2f8947e_row1774.jpg to D:/temp_fashion/
Moved 20231130235220_cce05c32_row1775.jpg to D:/temp_fashion/
Moved 20231130235221_8dd901c8_row1776.jpg to D:/temp_fashion/
Moved 20231130235222_1b2b6be7_row1777.jpg to D:/temp_fashion/
Moved 20231130235223_3507e665_row1778.jpg to D:/temp_fashion/
Moved 20231130235223_4d838cf9_row1779.jpg to D:/temp_fashion/
Moved 20231130235223_0bc69830_row1780.jpg to D:/temp_fashion/
Moved 20231130235224_88934a27_row1781.jpg to D:/temp_fashion/
Moved 20231130235225_688019b8_row1782.jpg to D:/temp_fashion/
Moved 20231130235225_64801968_row1783.jpg to D:/temp_fashion/
Moved 20231130235225_b146346d_row1784.jpg to D:/temp_fashion/
Moved 20231130235225_89a096c4_row1785.jpg to D:/temp_fashion/
Moved 20231130235226_8723aab7_row1786.jpg to D:/temp_fashion/
Moved 20

Moved 20231130235437_541df4a8_row2018.jpg to D:/temp_fashion/
Moved 20231130235438_895fe74d_row2019.jpg to D:/temp_fashion/
Moved 20231130235439_7fe88cf0_row2020.jpg to D:/temp_fashion/
Moved 20231130235440_fc25f336_row2021.jpg to D:/temp_fashion/
Moved 20231130235440_81a452a5_row2022.jpg to D:/temp_fashion/
Moved 20231130235440_5a333efd_row2023.jpg to D:/temp_fashion/
Moved 20231130235441_30695047_row2024.jpg to D:/temp_fashion/
Moved 20231130235441_3aa33807_row2025.jpg to D:/temp_fashion/
Moved 20231130235442_b81c50ae_row2026.jpg to D:/temp_fashion/
Moved 20231130235442_8cc6c0b5_row2027.jpg to D:/temp_fashion/
Moved 20231130235443_92a264cf_row2028.jpg to D:/temp_fashion/
Moved 20231130235443_2e42af33_row2029.jpg to D:/temp_fashion/
Moved 20231130235444_a1107510_row2030.jpg to D:/temp_fashion/
Moved 20231130235444_687eaf7a_row2031.jpg to D:/temp_fashion/
Moved 20231130235445_3f259a28_row2032.jpg to D:/temp_fashion/
Moved 20231130235445_25f564f2_row2033.jpg to D:/temp_fashion/
Moved 20

Moved 20231130235651_2a8bc1ce_row2279.jpg to D:/temp_fashion/
Moved 20231130235651_d855047b_row2280.jpg to D:/temp_fashion/
Moved 20231130235652_2152fca6_row2281.jpg to D:/temp_fashion/
Moved 20231130235653_0fc70d89_row2282.jpg to D:/temp_fashion/
Moved 20231130235653_64e37f9b_row2283.jpg to D:/temp_fashion/
Moved 20231130235655_f71df1e2_row2284.jpg to D:/temp_fashion/
Moved 20231130235655_ae39856a_row2285.jpg to D:/temp_fashion/
Moved 20231130235655_5d059360_row2286.jpg to D:/temp_fashion/
Moved 20231130235655_9d0bc708_row2287.jpg to D:/temp_fashion/
Moved 20231130235656_9a7d5a4e_row2288.jpg to D:/temp_fashion/
Moved 20231130235657_cb6babb1_row2289.jpg to D:/temp_fashion/
Moved 20231130235657_77b78834_row2290.jpg to D:/temp_fashion/
Moved 20231130235658_2fed5b2e_row2291.jpg to D:/temp_fashion/
Moved 20231130235659_34b63382_row2292.jpg to D:/temp_fashion/
Moved 20231130235659_d37e9806_row2293.jpg to D:/temp_fashion/
Moved 20231130235700_356b9c47_row2294.jpg to D:/temp_fashion/
Moved 20

Moved 20231130235927_ae3acca1_row2521.jpg to D:/temp_fashion/
Moved 20231130235927_6b95d5f2_row2522.jpg to D:/temp_fashion/
Moved 20231130235928_7a407b89_row2523.jpg to D:/temp_fashion/
Moved 20231130235929_90860774_row2524.jpg to D:/temp_fashion/
Moved 20231130235929_a47b4606_row2525.jpg to D:/temp_fashion/
Moved 20231130235929_6cbc3270_row2526.jpg to D:/temp_fashion/
Moved 20231130235930_0a7c2a3b_row2527.jpg to D:/temp_fashion/
Moved 20231130235931_64c7cbbc_row2528.jpg to D:/temp_fashion/
Moved 20231130235932_866383f8_row2529.jpg to D:/temp_fashion/
Moved 20231130235932_2eeac38b_row2530.jpg to D:/temp_fashion/
Moved 20231130235933_a9b947dc_row2531.jpg to D:/temp_fashion/
Moved 20231130235933_2ebeb8f1_row2532.jpg to D:/temp_fashion/
Moved 20231130235933_3752d730_row2533.jpg to D:/temp_fashion/
Moved 20231130235934_c21bb9a2_row2534.jpg to D:/temp_fashion/
Moved 20231130235935_34fc79cf_row2535.jpg to D:/temp_fashion/
Moved 20231130235936_1ab66934_row2536.jpg to D:/temp_fashion/
Moved 20

Moved 20231201000202_81586e5e_row2773.jpg to D:/temp_fashion/
Moved 20231201000202_cc381b1e_row2774.jpg to D:/temp_fashion/
Moved 20231201000203_3d2ddbde_row2775.jpg to D:/temp_fashion/
Moved 20231201000203_d9afbae2_row2776.jpg to D:/temp_fashion/
Moved 20231201000203_04f47e13_row2777.jpg to D:/temp_fashion/
Moved 20231201000204_4b3e9b81_row2778.jpg to D:/temp_fashion/
Moved 20231201000204_008da2a7_row2779.jpg to D:/temp_fashion/
Moved 20231201000205_d1640286_row2780.jpg to D:/temp_fashion/
Moved 20231201000205_e6a3c4a1_row2781.jpg to D:/temp_fashion/
Moved 20231201000206_40828e4b_row2782.jpg to D:/temp_fashion/
Moved 20231201000206_a6e374e0_row2783.jpg to D:/temp_fashion/
Moved 20231201000206_2502b326_row2784.jpg to D:/temp_fashion/
Moved 20231201000206_0364a2ba_row2785.jpg to D:/temp_fashion/
Moved 20231201000207_f8371207_row2786.jpg to D:/temp_fashion/
Moved 20231201000208_34e52fb5_row2787.jpg to D:/temp_fashion/
Moved 20231201000209_c4328c94_row2788.jpg to D:/temp_fashion/
Moved 20

...

Moved 20231201001857_9aaf89d0_row4649.jpg to D:/temp_fashion/
Moved 20231201001858_739a45c5_row4650.jpg to D:/temp_fashion/
Moved 20231201001858_da44fb16_row4651.jpg to D:/temp_fashion/
Moved 20231201001859_c3ef9cdd_row4652.jpg to D:/temp_fashion/
Moved 20231201001859_ff80f9f2_row4653.jpg to D:/temp_fashion/
Moved 20231201001859_d7f90a95_row4654.jpg to D:/temp_fashion/
Moved 20231201001900_2427d9a1_row4655.jpg to D:/temp_fashion/
Moved 20231201001901_17b37fcb_row4656.jpg to D:/temp_fashion/
Moved 20231201001901_e49fb507_row4657.jpg to D:/temp_fashion/
Moved 20231201001902_0874b6fb_row4658.jpg to D:/temp_fashion/
Moved 20231201001902_3ec5be37_row4659.jpg to D:/temp_fashion/
Moved 20231201001902_bb97d3cb_row4660.jpg to D:/temp_fashion/
Moved 20231201001903_3ab98ed7_row4661.jpg to D:/temp_fashion/
Moved 20231201001903_4bb75eb1_row4662.jpg to D:/temp_fashion/
Moved 20231201001904_091fd066_row4663.jpg to D:/temp_fashion/
Moved 20231201001904_698de019_row4664.jpg to D:/temp_fashion/
Moved 20

Moved 20231201002115_b8217469_row4952.jpg to D:/temp_fashion/
Moved 20231201002116_9d214856_row4953.jpg to D:/temp_fashion/
Moved 20231201002116_bf83aa63_row4954.jpg to D:/temp_fashion/
Moved 20231201002117_857cea7a_row4955.jpg to D:/temp_fashion/
Moved 20231201002117_9b927b53_row4956.jpg to D:/temp_fashion/
Moved 20231201002118_2cd7e7de_row4957.jpg to D:/temp_fashion/
Moved 20231201002119_8ee52a35_row4958.jpg to D:/temp_fashion/
Moved 20231201002119_27563435_row4959.jpg to D:/temp_fashion/
Moved 20231201002119_5b5cbc9a_row4960.jpg to D:/temp_fashion/
Moved 20231201002119_b2173b61_row4961.jpg to D:/temp_fashion/
Moved 20231201002120_4e026706_row4962.jpg to D:/temp_fashion/
Moved 20231201002120_b986df3f_row4963.jpg to D:/temp_fashion/
Moved 20231201002121_635851e8_row4964.jpg to D:/temp_fashion/
Moved 20231201002121_19682fbe_row4965.jpg to D:/temp_fashion/
Moved 20231201002121_6f1b931f_row4966.jpg to D:/temp_fashion/
Moved 20231201002122_7c9c6488_row4967.jpg to D:/temp_fashion/
Moved 20

Moved 20231201002338_8aee8815_row5234.jpg to D:/temp_fashion/
Moved 20231201002339_bdf275cc_row5235.jpg to D:/temp_fashion/
Moved 20231201002339_a3c739f9_row5236.jpg to D:/temp_fashion/
Moved 20231201002340_b9f3630b_row5237.jpg to D:/temp_fashion/
Moved 20231201002341_8997e082_row5238.jpg to D:/temp_fashion/
Moved 20231201002341_57dcf73b_row5239.jpg to D:/temp_fashion/
Moved 20231201002341_3c7fff4e_row5240.jpg to D:/temp_fashion/
Moved 20231201002341_863557de_row5241.jpg to D:/temp_fashion/
Moved 20231201002342_8137e2aa_row5242.jpg to D:/temp_fashion/
Moved 20231201002342_18331418_row5243.jpg to D:/temp_fashion/
Moved 20231201002343_72edd761_row5244.jpg to D:/temp_fashion/
Moved 20231201002343_e9720a2d_row5245.jpg to D:/temp_fashion/
Moved 20231201002343_0aa9e246_row5246.jpg to D:/temp_fashion/
Moved 20231201002344_76570277_row5247.jpg to D:/temp_fashion/
Moved 20231201002344_c69a2b64_row5248.jpg to D:/temp_fashion/
Moved 20231201002344_5dd79a75_row5249.jpg to D:/temp_fashion/
Moved 20

Moved 20231201002631_1ae4c3a6_row5558.jpg to D:/temp_fashion/
Moved 20231201002632_90dd7ffb_row5559.jpg to D:/temp_fashion/
Moved 20231201002632_79d249c0_row5560.jpg to D:/temp_fashion/
Moved 20231201002633_a749c21b_row5561.jpg to D:/temp_fashion/
Moved 20231201002633_24786f82_row5562.jpg to D:/temp_fashion/
Moved 20231201002633_1891fca3_row5563.jpg to D:/temp_fashion/
Moved 20231201002634_9bbc06c7_row5564.jpg to D:/temp_fashion/
Moved 20231201002635_6c06bf9f_row5565.jpg to D:/temp_fashion/
Moved 20231201002635_6c92b47a_row5566.jpg to D:/temp_fashion/
Moved 20231201002636_f9c9864e_row5567.jpg to D:/temp_fashion/
Moved 20231201002636_d81b58cb_row5568.jpg to D:/temp_fashion/
Moved 20231201002637_f4ca55fd_row5569.jpg to D:/temp_fashion/
Moved 20231201002638_1821f83d_row5570.jpg to D:/temp_fashion/
Moved 20231201002639_5e85fa70_row5571.jpg to D:/temp_fashion/
Moved 20231201002639_6d33738e_row5572.jpg to D:/temp_fashion/
Moved 20231201002640_9910964a_row5573.jpg to D:/temp_fashion/
Moved 20

Moved 20231201002925_27919842_row5902.jpg to D:/temp_fashion/
Moved 20231201002926_a9f8c408_row5903.jpg to D:/temp_fashion/
Moved 20231201002927_57135ef8_row5904.jpg to D:/temp_fashion/
Moved 20231201002927_0372a8e6_row5905.jpg to D:/temp_fashion/
Moved 20231201002927_241737cb_row5906.jpg to D:/temp_fashion/
Moved 20231201002928_1a8b2a26_row5907.jpg to D:/temp_fashion/
Moved 20231201002929_3298a8c6_row5908.jpg to D:/temp_fashion/
Moved 20231201002929_571c7966_row5909.jpg to D:/temp_fashion/
Moved 20231201002929_ea532abb_row5910.jpg to D:/temp_fashion/
Moved 20231201002929_504753d3_row5911.jpg to D:/temp_fashion/
Moved 20231201002930_18ff24ef_row5912.jpg to D:/temp_fashion/
Moved 20231201002930_b21236af_row5913.jpg to D:/temp_fashion/
Moved 20231201002930_eb738015_row5914.jpg to D:/temp_fashion/
Moved 20231201002931_d4277efe_row5915.jpg to D:/temp_fashion/
Moved 20231201002932_e38d669a_row5916.jpg to D:/temp_fashion/
Moved 20231201002933_cc5d1ae0_row5917.jpg to D:/temp_fashion/
Moved 20

In [78]:
img = image.load_img('samples/coat5.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

neighbors = NearestNeighbors(n_neighbors=10,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

distances,indices = neighbors.kneighbors([normalized_result])

print(indices)

for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)

[[2636 2677 2663 2359 2638 2624 2601 2672 2608 2670]]
